In [1]:
import pandas as pd
import numpy as np
import random
import warnings

warnings.filterwarnings("ignore")

In [2]:
# Load the Excel file
appointment_df = pd.read_excel("Jun 10 Davey Data Send to Fiverr.xlsx", sheet_name='Appointments')
lookup_df = pd.read_excel("2024.08.21 GPS Matching Logic For Fiverr.xlsx", sheet_name='Employee-Vehicle Match')
gps_df = pd.read_excel("Jun 10 Davey Data Send to Fiverr.xlsx", sheet_name='Breadcrumb-Davey')

lookup_df = lookup_df[["Employee", "ID", "Active", "Most Recent Visit", "Vehicle/Asset"]]

In [3]:
# Filter the appointment table to include only rows where status = 1
appointment_df = appointment_df[appointment_df['status'] == 1]

In [4]:
# Rename columns in the lookup and GPS tables for consistency
lookup_df.rename(columns={'ID': 'EID', 'Vehicle/Asset': 'vehicle_name'}, inplace=True)
gps_df.rename(columns={'Name': 'vehicle_name'}, inplace=True)

In [5]:
# Create a new column 'EID' in the appointment table derived from the 'servicedBy' column
appointment_df['EID'] = appointment_df['servicedBy']

In [6]:
# Ensure consistent data types
appointment_df['EID'] = appointment_df['EID'].astype("int64")
appointment_df['Latitude'] = appointment_df['Latitude'].astype(float)
appointment_df['Longitude'] = appointment_df['Longitude'].astype(float)
appointment_df['duration'] = appointment_df['duration'].astype(float)
lookup_df['EID'] = lookup_df['EID'].astype("int64")
lookup_df['vehicle_name'] = lookup_df['vehicle_name'].astype(str)
gps_df['vehicle_name'] = gps_df['vehicle_name'].astype(str)

In [7]:
# Split lat and lon from Location column in GPS table
gps_df[["lat", "lon"]] = gps_df["Location"].str.split(",", expand=True).astype(float)

In [8]:
# Perform a left merge of the appointment table with the lookup table on EID
merged_df = pd.merge(appointment_df, lookup_df[['EID', 'vehicle_name']], on='EID', how='left')

# Handle missing matches by assigning "unknown vehicle" to rows where vehicle_name is NaN
merged_df['vehicle_name'].fillna('unknown vehicle', inplace=True)

# Count the number of "unknown vehicle" entries
unknown_vehicle_count = merged_df[merged_df['vehicle_name'] == 'unknown vehicle'].shape[0]

# Log the count of unknown vehicles
print(f"Number of appointments with 'unknown vehicle': {unknown_vehicle_count}")

Number of appointments with 'unknown vehicle': 0


In [9]:
# Add a new column 'geofence' to the appointment table
# Set 'geofence' to the default value (0.1 miles) for all appointments
merged_df['geofence'] = 0.1

In [10]:
# Initialize a dictionary to store filtered and ordered GPS tables for each vehicle_name
filtered_gps_tables = {}

# Get the unique vehicle names from the appointment table
unique_vehicle_names = merged_df['vehicle_name'].unique()

# For each unique 'vehicle_name' in the appointment table:
for vehicle_name in unique_vehicle_names:
    # Filter the GPS table to include only rows where 'vehicle_name' matches
    filtered_gps = gps_df[gps_df['vehicle_name'] == vehicle_name]
    
    # Order the filtered GPS table by 'Day_and_Time', with earlier times first
    filtered_gps = filtered_gps.sort_values(by='Time', ascending=True)
    
    # Store the filtered and ordered table in the dictionary
    filtered_gps_tables[vehicle_name] = filtered_gps

In [11]:
# Function to handle consecutive "Ignition On" and "Ignition Off" entries
def remove_redundant_ignition_events(gps_data):
    # Shift the Event Type column to compare consecutive rows
    gps_data['prev_event'] = gps_data['Event Type'].shift(-1)
    
    # Create a mask to identify redundant "Ignition On" and "Ignition Off" events
    redundant_mask = (
        ((gps_data['Event Type'] == 'Ignition On') & (gps_data['prev_event'] == 'Ignition On')) |
        ((gps_data['Event Type'] == 'Ignition Off') & (gps_data['prev_event'] == 'Ignition Off'))
    )
    
    # Remove the redundant events
    refined_gps_data = gps_data[~redundant_mask].copy()
    
    # Drop the temporary 'prev_event' column
    refined_gps_data.drop(columns=['prev_event'], inplace=True)
    
    return refined_gps_data

# For each filtered and ordered GPS table:
for vehicle_name, gps_data in filtered_gps_tables.items():
    # Apply the function to remove redundant "Ignition On" and "Ignition Off" entries
    filtered_gps_tables[vehicle_name] = remove_redundant_ignition_events(gps_data)

In [12]:
# Function to calculate Euclidean distance between two points (latitude, longitude)
def calculate_distance(lat1, lon1, lat2, lon2):
    return np.sqrt(((69.1 * (lat1 - lat2))**2) + ((69.1 * (lon1 - lon2))**2))

# Adding "in the zone" flag to GPS entries based on geofence
def flag_in_the_zone(gps_data, appointment_lat, appointment_lon, geofence_radius):
    # Calculate the distance between each GPS entry and the appointment's location
    gps_data['distance_to_appointment'] = calculate_distance(gps_data['lat'], gps_data['lon'], appointment_lat, appointment_lon)
    
    # Flag as "in the zone" if the distance is within the geofence
    gps_data['in_the_zone'] = gps_data['distance_to_appointment'] < geofence_radius
    
    return gps_data

# Creating the nested dictionary to store results
flagged_gps_data = {}

# Loop through each appointment and its corresponding GPS data
for index, appointment in merged_df.iterrows():
    vehicle_name = appointment['vehicle_name']
    appointment_id = appointment['appointmentID']
    
    # Get the corresponding GPS data for this vehicle
    if vehicle_name in filtered_gps_tables:
        gps_data = filtered_gps_tables[vehicle_name]
        if not gps_data.empty:
            # Calculate the Euclidean distance for each GPS entry
            appointment_lat = appointment['Latitude']
            appointment_lon = appointment['Longitude']
            geofence_radius = appointment['geofence']  # assuming geofence is in miles

            # Flag "in the zone" entries
            flagged_gps_data_temp = flag_in_the_zone(gps_data.copy(), appointment_lat, appointment_lon, geofence_radius)
        
            flagged_gps_data[appointment_id] = flagged_gps_data_temp


In [13]:
serviced_flagged_gps_data = {}

# Function to flag GPS entries as "servicing the zone"
def flag_servicing_the_zone(gps_data):
    # Initialize the "servicing the zone" flag as False
    gps_data['servicing_the_zone'] = False

    # Create masks for the conditions where "servicing the zone" should be True
    ignition_mask = (
        (gps_data['in_the_zone']) & 
        (gps_data['Event Type'].isin(['Ignition Off', 'Ignition On']))
    )

    gps_message_mask = (
        (gps_data['in_the_zone']) &
        (gps_data['Event Type'] == 'Gps Message') &
        (
            (gps_data['in_the_zone'].shift(1)) |  # Check the previous row
            (gps_data['in_the_zone'].shift(-1))   # Check the next row
        )
    )

    # Set "servicing the zone" to True where either condition is met
    gps_data.loc[ignition_mask | gps_message_mask, 'servicing_the_zone'] = True

    return gps_data

# Loop through each GPS table and apply the "servicing the zone" flagging
for appointment_id, gps_data in flagged_gps_data.items():
    serviced_flagged_gps_data[appointment_id] = flag_servicing_the_zone(gps_data.copy())

In [14]:
# Function to identify ZoneStartTime and ZoneEndTime
def identify_zone_times(gps_data):
    # Initialize ZoneStartTime and ZoneEndTime
    zone_start_times = []
    zone_end_times = []

    # Shift the "servicing the zone" column to compare consecutive rows
    gps_data['prev_servicing'] = gps_data['servicing_the_zone'].shift(1, fill_value=False)
    gps_data['next_servicing'] = gps_data['servicing_the_zone'].shift(-1, fill_value=False)
    
    # Identify ZoneStartTime: When "servicing the zone" changes from False to True
    start_time_mask = (gps_data['servicing_the_zone'] == True) & (gps_data['prev_servicing'] == False)
    zone_start_times = gps_data.loc[start_time_mask, 'Time'].tolist()

    # Identify ZoneEndTime: When "servicing the zone" changes from True to False
    end_time_mask = (gps_data['servicing_the_zone'] == True) & (gps_data['next_servicing'] == False)
    zone_end_times = gps_data.loc[end_time_mask, 'Time'].tolist()
    
    return zone_start_times, zone_end_times

merged_df["ZoneStartTime"] = None
merged_df["ZoneEndTime"] = None

# Loop through each appointment and corresponding GPS table to identify ZoneStartTime and ZoneEndTime
for index, appointment in merged_df.iterrows():
    appointment_id = appointment['appointmentID']
    
    if appointment_id in serviced_flagged_gps_data:
        gps_data = serviced_flagged_gps_data[appointment_id]
        
        if not gps_data.empty:
            # Identify ZoneStartTime and ZoneEndTime for the current appointment's GPS data
            zone_start_time, zone_end_time = identify_zone_times(gps_data.copy())

            # Update the appointment table with the identified times
            merged_df.at[index, 'ZoneStartTime'] = zone_start_time
            merged_df.at[index, 'ZoneEndTime'] = zone_end_time

In [15]:
merged_df["Formatted_TimeIN"] = pd.to_datetime(merged_df["date"].astype(str) + ' ' + merged_df["start"].astype(str))

In [16]:
# Function to confirm start and end times with error handling
def confirm_start_end_times(appointment, formatted_time_in):
    zone_start_times = appointment['ZoneStartTime']
    zone_end_times = appointment['ZoneEndTime']
    
    confirmed_start_time = None
    confirmed_end_time = None
    
    if zone_start_times and len(zone_start_times) == 1:
        # If there is only one ZoneStartTime and ZoneEndTime, use them
        confirmed_start_time = zone_start_times[0]
        confirmed_end_time = zone_end_times[0]
    elif zone_start_times and len(zone_start_times) > 1:
        # Log this as an error due to multiple start times
        print(f"Error: Multiple ZoneStartTime entries found for appointmentID {appointment['appointmentID']}")
        
        
        # Find the ZoneStartTime closest to Formatted_TimeIN
        closest_start_time = min(zone_start_times, key=lambda x: abs(pd.to_datetime(x) - formatted_time_in))
        
        # Set ConfirmedStartTime to the closest ZoneStartTime and the corresponding ZoneEndTime
        confirmed_start_time = closest_start_time
        closest_index = zone_start_times.index(closest_start_time)
        confirmed_end_time = zone_end_times[closest_index]
    else:
        # Log this as an error if no confirmed times can be set
        print(f"Error: No ZoneStartTime entries found for appointmentID {appointment['appointmentID']}")
    
    return confirmed_start_time, confirmed_end_time

# Add new columns for ConfirmedStartTime and ConfirmedEndTime
merged_df["GPSStartTime"] = None
merged_df["GPSEndTime"] = None

# Loop through each appointment to confirm start and end times
for index, appointment in merged_df.iterrows():
    formatted_time_in = appointment['Formatted_TimeIN']
    
    # Confirm start and end times with error handling
    confirmed_start_time, confirmed_end_time = confirm_start_end_times(appointment.copy(), formatted_time_in)
    
    # Update the appointment table with the confirmed times
    merged_df.at[index, 'GPSStartTime'] = confirmed_start_time
    merged_df.at[index, 'GPSEndTime'] = confirmed_end_time


Error: No ZoneStartTime entries found for appointmentID 377867
Error: No ZoneStartTime entries found for appointmentID 378549
Error: No ZoneStartTime entries found for appointmentID 385187
Error: No ZoneStartTime entries found for appointmentID 380043
Error: No ZoneStartTime entries found for appointmentID 382248
Error: No ZoneStartTime entries found for appointmentID 378551
Error: No ZoneStartTime entries found for appointmentID 379814
Error: No ZoneStartTime entries found for appointmentID 377869
Error: No ZoneStartTime entries found for appointmentID 382251
Error: No ZoneStartTime entries found for appointmentID 373121
Error: No ZoneStartTime entries found for appointmentID 380037
Error: No ZoneStartTime entries found for appointmentID 385922
Error: No ZoneStartTime entries found for appointmentID 384604
Error: No ZoneStartTime entries found for appointmentID 377868
Error: No ZoneStartTime entries found for appointmentID 373133
Error: No ZoneStartTime entries found for appointmentID

In [17]:
merged_df["GPSStartTime"] = pd.to_datetime(merged_df["GPSStartTime"])
merged_df["GPSEndTime"] = pd.to_datetime(merged_df["GPSEndTime"])

In [18]:
merged_df.rename(columns={"duration": "Duration", "timeIn": "CRMTimeIn", "timeOut": "CRMTimeOut",
                          "checkIn" : "CheckIn", "checkOut": "CheckOut"}, inplace=True)

In [19]:
merged_df["GPSTime"] = np.round((merged_df["GPSEndTime"] - merged_df["GPSStartTime"]).apply(lambda x: x.total_seconds()) / 60, 2)
merged_df["CRMTime"] = np.round((merged_df["CRMTimeOut"] - merged_df["CRMTimeIn"]).apply(lambda x: x.total_seconds()) / 60, 2)
merged_df["DurationVs.Mobile"] = merged_df["CRMTime"] > (2*merged_df["Duration"])
merged_df["FinalCRMTime"] = [f["Duration"] if f["DurationVs.Mobile"] else f["CRMTime"] for i,f in merged_df.iterrows()]

In [20]:
final_output = (merged_df[["appointmentID", "vehicle_name", "Address", "GPSStartTime", "GPSEndTime", "GPSTime", 
                           "CRMTimeIn", "CRMTimeOut", "CheckIn", "CheckOut", "CRMTime", "Duration",
                           "DurationVs.Mobile", "FinalCRMTime"]]
                .dropna()
                .sort_values("GPSStartTime")
                .reset_index())

final_output.rename(columns={"index": "Appointment#"}, inplace=True)
final_output["Appointment#"] = range(1,final_output.shape[0]+1)

In [21]:
final_output

,Appointment#,appointmentID,vehicle_name,Address,GPSStartTime,GPSEndTime,GPSTime,CRMTimeIn,CRMTimeOut,CheckIn,CheckOut,CRMTime,Duration,DurationVs.Mobile,FinalCRMTime
0,1,373129,Davey #24,110 US Hwy 64 Cashiers NC 28717,2024-06-10 07:57:00,2024-06-10 08:14:10,17.17,2024-06-10 07:56:31,2024-06-10 08:11:50,2024-06-10 04:56:31,2024-06-10 05:11:51,15.32,30.0,False,15.32
1,2,384076,Davey #24,111 Headwater Dr. Glenville NC 28736,2024-06-10 08:28:10,2024-06-10 08:42:55,14.75,2024-06-10 08:28:38,2024-06-10 08:40:38,2024-06-10 05:28:39,2024-06-10 05:40:39,12.00,45.0,False,12.00
2,3,385038,Davey #24,356 Haven Ridge Trl (Cedar Creek Woods) Cashie...,2024-06-10 08:48:55,2024-06-10 09:28:31,39.60,2024-06-10 08:50:28,2024-06-10 09:26:11,2024-06-10 05:50:29,2024-06-10 06:26:12,35.72,45.0,False,35.72
3,4,378412,Davey #24,196 Burns Street Cashiers NC 28717,2024-06-10 09:38:02,2024-06-10 09:53:44,15.70,2024-06-10 09:37:22,2024-06-10 09:49:54,2024-06-10 06:37:22,2024-06-10 06:49:55,12.53,30.0,False,12.53
4,5,378411,Davey #24,53 Cottage Walk Ln. Cashiers NC 28717,2024-06-10 09:38:02,2024-06-10 10:08:05,30.05,2024-06-10 09:57:43,2024-06-10 10:07:33,2024-06-10 06:57:43,2024-06-10 07:07:34,9.83,30.0,False,9.83
5,6,373141,Davey #24,18 Cashiers School Rd. Cashiers NC 28717,2024-06-10 10:11:51,2024-06-10 10:36:40,24.82,2024-06-10 10:11:23,2024-06-10 10:35:45,2024-06-10 07:11:23,2024-06-10 07:35:46,24.37,30.0,False,24.37
6,7,373135,Davey #24,181 Hwy 64 W Suite 1 Cashiers NC 28717,2024-06-10 10:40:05,2024-06-10 11:08:55,28.83,2024-06-10 10:41:40,2024-06-10 11:07:13,2024-06-10 07:41:40,2024-06-10 08:07:14,25.55,30.0,False,25.55
7,8,373174,Davey #24,155 Foxfire Road Cashiers NC 28717,2024-06-10 11:24:35,2024-06-10 11:41:46,17.18,2024-06-10 11:23:52,2024-06-10 11:41:15,2024-06-10 08:23:53,2024-06-10 08:41:17,17.38,30.0,False,17.38
8,9,372561,Davey #24,75 High Springs Cashiers NC 28717,2024-06-10 11:53:46,2024-06-10 12:22:36,28.83,2024-06-10 11:53:44,2024-06-10 23:24:44,0000-00-00 00:00:00,2024-06-10 09:32:39,691.00,30.0,True,30.00
9,10,378397,Davey #24,35 Top of the Ridge Rd Cashiers NC 28717,2024-06-10 12:44:15,2024-06-10 12:57:44,13.48,2024-06-10 12:43:54,2024-06-10 12:56:51,2024-06-10 09:43:55,2024-06-10 09:56:52,12.95,30.0,False,12.95


In [22]:
final_output.to_excel("FinalOutput.xlsx", index=False)